In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
from tqdm import tqdm
import os, re, time, random
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import pandas as pd
from functools import reduce
import inspect
from selenium.webdriver.common.keys import Keys

# Analyze Pandas usage in Kaggle Kernels

## 1. Get urls of top kernels by vote

In [48]:
browser = webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver')

browser.get("https://www.kaggle.com/kernels?sortBy=votes&group=everyone&pageSize=20&language=Python")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 100

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

soup = BeautifulSoup(browser.page_source, "html.parser")

KeyboardInterrupt: 

In [23]:
kernel_links = ['https://www.kaggle.com/' + a['href'] + '/code' for a in soup.find_all("a", class_="block-link__anchor") if ('teaching-notebook-for-total-imaging-newbies' not in a['href'] and 'mentions-of-kaggle-on-hacker-news' not in a['href'])]

In [24]:
len(kernel_links)

1479

## 2. Get url of code files from kernels page

In [72]:
THREADS = 1
browsers = [webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver') for _ in range(THREADS)]
pool = ThreadPool(THREADS)
blacklist = ['https://www.kaggle.com/stkbailey/teaching-notebook-for-total-imaging-newbies/code']

In [67]:
def get_code_from_kernel(kernel_link):
    if 'stkbailey' in kernel_link or 'mentions-of-kaggle-on-hacker-news' in kernel_link:
        return None
    browser = browsers[random.randint(0, THREADS - 1)]
    try:
        browser.get(kernel_link)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML, 'html.parser')

        link = soup.find_all("a", class_="script-code-pane__download")[0]['href']
        
        return link
    except Exception as e:
        print("E", e)
        if 'timeout' in str(e):
            return str(e)
        return None

In [34]:
code_links_write_part1 = []
for kl in tqdm(kernel_links[:300]):
    code_links_write_part1.append(get_code_from_kernel(kl))
    
code_file_write_part1 = open('code_links_part1.txt', 'w')
for code_link in code_links_write_part1:
    code_file_write_part1.write("%s\n" % code_link)


100%|██████████| 300/300 [21:55<00:00,  4.38s/it]


In [35]:
code_links_write_part2 = []
for kl in tqdm(kernel_links[300:600]):
    code_links_write_part2.append(get_code_from_kernel(kl))
    
code_file_write_part2 = open('code_links_part2.txt', 'w')
for code_link in code_links_write_part2:
    code_file_write_part2.write("%s\n" % code_link)

100%|██████████| 300/300 [15:15<00:00,  3.05s/it]


In [36]:
code_links_write_part3 = []
for kl in tqdm(kernel_links[600:900]):
    code_links_write_part3.append(get_code_from_kernel(kl))
    
code_file_write_part3 = open('code_links_part3.txt', 'w')
for code_link in code_links_write_part3:
    code_file_write_part3.write("%s\n" % code_link)

 51%|█████     | 152/300 [06:20<06:10,  2.50s/it]

E list index out of range


 65%|██████▌   | 196/300 [08:03<04:16,  2.47s/it]

E list index out of range


100%|██████████| 300/300 [13:21<00:00,  2.67s/it]


In [70]:
code_links_write_part4 = []
for kl in tqdm(kernel_links[900:1050]):
    code_links_write_part4.append(get_code_from_kernel(kl))
    
code_file_write_part4 = open('code_links_part4.txt', 'w')
for code_link in code_links_write_part4:
    code_file_write_part4.write("%s\n" % code_link)

 50%|█████     | 75/150 [03:54<03:54,  3.13s/it]

E list index out of range


100%|██████████| 150/150 [07:18<00:00,  2.92s/it]


In [74]:
code_links_write_part5 = []
for kl in tqdm(kernel_links[1050:1200]):
    if 'keras-rcnn-based-overview-wip' not in kl:
        res = get_code_from_kernel(kl)
        if res is not None and 'timeout' in res:
            browsers = [webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver') for _ in range(THREADS)]
        else:
            code_links_write_part5.append(res)
    
code_file_write_part5 = open('code_links_part5.txt', 'w')
for code_link in code_links_write_part5:
    code_file_write_part5.write("%s\n" % code_link)


  4%|▍         | 6/150 [00:13<05:24,  2.25s/it]

E list index out of range


100%|██████████| 150/150 [06:34<00:00,  2.63s/it]


In [69]:
code_links_write_part6 = []
for kl in tqdm(kernel_links[1200:1500]):
    code_links_write_part6.append(get_code_from_kernel(kl))
    
code_file_write_part6 = open('code_links_part6.txt', 'w')
for code_link in code_links_write_part6:
    code_file_write_part6.write("%s\n" % code_link)

 18%|█▊        | 49/279 [01:58<09:18,  2.43s/it]

E list index out of range


 34%|███▎      | 94/279 [03:50<07:34,  2.46s/it]

E list index out of range


 34%|███▍      | 95/279 [03:51<07:28,  2.44s/it]

E list index out of range


100%|██████████| 279/279 [11:53<00:00,  2.56s/it]


In [76]:
code_links_read = []
for i in range(1, 7):
    with open('code_links_part' + str(i) + '.txt', 'r') as code_file_read:
        code_links_read += code_file_read.read().splitlines()

In [77]:
code_links = ['https://www.kaggle.com' + cl for cl in code_links_read]
code_links[:5]

['https://www.kaggle.com/kernels/scriptcontent/3123055/download',
 'https://www.kaggle.com/kernels/scriptcontent/2368078/download',
 'https://www.kaggle.com/kernels/scriptcontent/928737/download',
 'https://www.kaggle.com/kernels/scriptcontent/3253516/download',
 'https://www.kaggle.com/kernels/scriptcontent/2031417/download']

## 3. Pull raw code file to local dir

In [78]:
for code_link in tqdm(code_links):
    !wget --content-disposition $code_link -P data

  0%|          | 0/983 [00:00<?, ?it/s]

--2018-04-17 22:40:35--  https://www.kaggle.com/kernels/scriptcontent/3123055/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48677 (48K) [application/octet-stream]
Saving to: ‘data/script.ipynb’

script.ipynb        100%[===================>]  47.54K  --.-KB/s    in 0.06s   

2018-04-17 22:40:36 (734 KB/s) - ‘data/script.ipynb’ saved [48677/48677]



  0%|          | 1/983 [00:00<12:48,  1.28it/s]

--2018-04-17 22:40:36--  https://www.kaggle.com/kernels/scriptcontent/2368078/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237024 (231K) [application/octet-stream]
Saving to: ‘data/script.ipynb.1’

script.ipynb.1      100%[===================>] 231.47K  1.01MB/s    in 0.2s    

2018-04-17 22:40:37 (1.01 MB/s) - ‘data/script.ipynb.1’ saved [237024/237024]



  0%|          | 2/983 [00:01<12:46,  1.28it/s]

...

100%|█████████▉| 981/983 [08:31<00:01,  1.92it/s]

--2018-04-17 22:49:07--  https://www.kaggle.com/kernels/scriptcontent/1570700/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19792 (19K) [application/octet-stream]
Saving to: ‘data/script.ipynb.681’

script.ipynb.681    100%[===================>]  19.33K  --.-KB/s    in 0.05s   

2018-04-17 22:49:12 (364 KB/s) - ‘data/script.ipynb.681’ saved [19792/19792]



100%|█████████▉| 982/983 [08:36<00:00,  1.90it/s]

--2018-04-17 22:49:12--  https://www.kaggle.com/kernels/scriptcontent/1432803/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74198 (72K) [application/octet-stream]
Saving to: ‘data/script.ipynb.682’

script.ipynb.682    100%[===================>]  72.46K  --.-KB/s    in 0.1s    

2018-04-17 22:49:13 (608 KB/s) - ‘data/script.ipynb.682’ saved [74198/74198]



100%|██████████| 983/983 [08:37<00:00,  1.90it/s]


## 4. Clean non-Python files from downloaded

In [79]:
for filename in os.listdir("data"):
    if filename == ".DS_Store":
        pass
    
    name_parts = filename.split(".")
    if len(name_parts) == 2:
        name_parts.append("0")
        
    if name_parts[1] in ['ipynb', 'py']:
        new_name = name_parts[0] + name_parts[2] + "." + name_parts[1]
        new_name = 'r' + new_name if name_parts[1] == 'py' else new_name
        os.rename('data/' + filename, 'data/' + new_name)

    else:
        os.remove('data/' + filename)

## Tokenize Notebooks and Scripts

In [80]:
# Analyze python script
pd_search_tokens = ["." + p[0] for p in inspect.getmembers(pd) if p[1].__class__.__name__ == 'function'] + ['.DataFrame']
df_search_tokens = ["." + p[0] for p in inspect.getmembers(pd.DataFrame) if p[1].__class__.__name__ == 'function']

def parse_script(script_name):
    with open(script_name) as script_file:
        code = script_file.read().splitlines()

    ps = {}
    for line in code:
        for token in pd_search_tokens:
            ct = line.count(token)
            if ct > 0 and token not in ps:
                ps['pd' + token] = ct
            elif ct > 0:
                ps['pd' + token] += ct
    
        for token in df_search_tokens:
            ct = line.count(token)
            if ct > 0 and token not in ps:
                ps['df' + token] = ct
            elif ct > 0:
                ps['df' + token] += ct
    
    return ps

In [82]:
all_methods = []
for filename in os.listdir("data"):
    if filename.split(".")[1] == 'py':
        all_methods.append(parse_script('data/' + filename))
    else:
        pfilename = 'data/' + filename
        cfilename = filename.split(".")[0] + '.py'
        if cfilename not in os.listdir("data"):
            !jupyter nbconvert --to=python $pfilename
            time.sleep(1)
        try:
            all_methods.append(parse_script('data/' + cfilename))
        except:
            pass

[NbConvertApp] Converting notebook data/script0.ipynb to python
[NbConvertApp] Writing 32874 bytes to data/script0.py
[NbConvertApp] Converting notebook data/script1.ipynb to python
[NbConvertApp] Writing 29921 bytes to data/script1.py
[NbConvertApp] Converting notebook data/script10.ipynb to python
[NbConvertApp] Writing 5677 bytes to data/script10.py
[NbConvertApp] Converting notebook data/script100.ipynb to python
[NbConvertApp] Writing 20321 bytes to data/script100.py
[NbConvertApp] Converting notebook data/script101.ipynb to python
[NbConvertApp] Writing 5987 bytes to data/script101.py
[NbConvertApp] Converting notebook data/script102.ipynb to python
[NbConvertApp] Writing 3724 bytes to data/script102.py
[NbConvertApp] Converting notebook data/script103.ipynb to python
[NbConvertApp] Writing 7629 bytes to data/script103.py
[NbConvertApp] Converting notebook data/script104.ipynb to python
[NbConvertApp] Writing 13238 bytes to data/script104.py
[NbConvertApp] Converting notebook dat

[NbConvertApp] Writing 69707 bytes to data/script139.py
[NbConvertApp] Converting notebook data/script14.ipynb to python
[NbConvertApp] Writing 42766 bytes to data/script14.py
[NbConvertApp] Converting notebook data/script140.ipynb to python
[NbConvertApp] Writing 10367 bytes to data/script140.py
[NbConvertApp] Converting notebook data/script141.ipynb to python
[NbConvertApp] Writing 117648 bytes to data/script141.py
[NbConvertApp] Converting notebook data/script142.ipynb to python
[NbConvertApp] Writing 17212 bytes to data/script142.py
[NbConvertApp] Converting notebook data/script143.ipynb to python
[NbConvertApp] Writing 5026 bytes to data/script143.py
[NbConvertApp] Converting notebook data/script144.ipynb to python
[NbConvertApp] Writing 15772 bytes to data/script144.py
[NbConvertApp] Converting notebook data/script145.ipynb to python
[NbConvertApp] Writing 36497 bytes to data/script145.py
[NbConvertApp] Converting notebook data/script146.ipynb to python
[NbConvertApp] Writing 881

[NbConvertApp] Converting notebook data/script18.ipynb to python
[NbConvertApp] Writing 4265 bytes to data/script18.py
[NbConvertApp] Converting notebook data/script180.ipynb to python
[NbConvertApp] Writing 4710 bytes to data/script180.py
[NbConvertApp] Converting notebook data/script181.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '04577c7b-a97d-4676-ba2d-2af26e58d762',
              '_execution_state': 'idle',
              '_uuid': '3e17a45b006b3fbcc550284747e278ae71908e3e'},
 'outputs': ['...0 outputs...'],
 'source': '# Getting Started with the NIPS 2017 Adversarial Learning '
           'Challe...'}
[NbConvertApp] Writing 11504 bytes to data/script181.py
[NbConvertApp] Converting notebook data/script182.ipynb to 

[NbConvertApp] Writing 5307 bytes to data/script217.py
[NbConvertApp] Converting notebook data/script218.ipynb to python
[NbConvertApp] Writing 5545 bytes to data/script218.py
[NbConvertApp] Converting notebook data/script219.ipynb to python
[NbConvertApp] Writing 7637 bytes to data/script219.py
[NbConvertApp] Converting notebook data/script22.ipynb to python
[NbConvertApp] Writing 77854 bytes to data/script22.py
[NbConvertApp] Converting notebook data/script220.ipynb to python
[NbConvertApp] Writing 15318 bytes to data/script220.py
[NbConvertApp] Converting notebook data/script221.ipynb to python
[NbConvertApp] Writing 7286 bytes to data/script221.py
[NbConvertApp] Converting notebook data/script222.ipynb to python
[NbConvertApp] Writing 30409 bytes to data/script222.py
[NbConvertApp] Converting notebook data/script223.ipynb to python
[NbConvertApp] Writing 21197 bytes to data/script223.py
[NbConvertApp] Converting notebook data/script224.ipynb to python
[NbConvertApp] Writing 51450 b

[NbConvertApp] Writing 5561 bytes to data/script258.py
[NbConvertApp] Converting notebook data/script259.ipynb to python
[NbConvertApp] Writing 4937 bytes to data/script259.py
[NbConvertApp] Converting notebook data/script26.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '2f790bbf-1719-4d91-977c-c0890ac78192',
              '_uuid': 'd078acf4cc246329c1107222b8ba73761a9b42b5'},
 'outputs': ['...0 outputs...'],
 'source': '# Titanic Top 4% with ensemble modeling\n'
           '### **Yassine Ghouzam, P...'}
[NbConvertApp] Writing 32986 bytes to data/script26.py
[NbConvertApp] Converting notebook data/script260.ipynb to python
[NbConvertApp] Writing 20064 bytes to data/script260.py
[NbConvertApp] Converting notebook data/scri

...

[NbConvertApp] Writing 16639 bytes to data/script507.py
[NbConvertApp] Converting notebook data/script508.ipynb to python
[NbConvertApp] Writing 26907 bytes to data/script508.py
[NbConvertApp] Converting notebook data/script509.ipynb to python
[NbConvertApp] Writing 2651 bytes to data/script509.py
[NbConvertApp] Converting notebook data/script51.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': 'aac1eb7a-6eb0-4239-bf0c-7edf5dd242a1',
              '_uuid': '56f92656487e77f60088d9afe9969a878ac1ead5'},
 'outputs': ['...0 outputs...'],
 'source': '<table>\n'
           '    <tr>\n'
           '        <td>\n'
           '        <center>\n'
           '        <font siz...'}
[NbConvertApp] Writing 6090 bytes to data/script51.py

[NbConvertApp] Writing 22978 bytes to data/script541.py
[NbConvertApp] Converting notebook data/script542.ipynb to python
[NbConvertApp] Writing 7909 bytes to data/script542.py
[NbConvertApp] Converting notebook data/script543.ipynb to python
[NbConvertApp] Writing 4432 bytes to data/script543.py
[NbConvertApp] Converting notebook data/script544.ipynb to python
[NbConvertApp] Writing 49035 bytes to data/script544.py
[NbConvertApp] Converting notebook data/script545.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': 'd894877c-753f-4947-a6a0-8c100b8af6b2',
              '_uuid': '3a208d285d49bbe7c35827de8416e3b7c5c061ae'},
 'outputs': ['...0 outputs...'],
 'source': '## Import library'}
[NbConvertApp] Writing 4103 bytes to data

[NbConvertApp] Writing 3351 bytes to data/script580.py
[NbConvertApp] Converting notebook data/script581.ipynb to python
[NbConvertApp] Writing 19349 bytes to data/script581.py
[NbConvertApp] Converting notebook data/script582.ipynb to python
[NbConvertApp] Writing 127696 bytes to data/script582.py
[NbConvertApp] Converting notebook data/script583.ipynb to python
[NbConvertApp] Writing 2092 bytes to data/script583.py
[NbConvertApp] Converting notebook data/script584.ipynb to python
[NbConvertApp] Writing 3331 bytes to data/script584.py
[NbConvertApp] Converting notebook data/script585.ipynb to python
[NbConvertApp] Writing 5792 bytes to data/script585.py
[NbConvertApp] Converting notebook data/script586.ipynb to python
[NbConvertApp] Writing 6638 bytes to data/script586.py
[NbConvertApp] Converting notebook data/script587.ipynb to python
[NbConvertApp] Writing 884 bytes to data/script587.py
[NbConvertApp] Converting notebook data/script588.ipynb to python
[NbConvertApp] Writing 4124 by

[NbConvertApp] Writing 3850 bytes to data/script63.py
[NbConvertApp] Converting notebook data/script630.ipynb to python
[NbConvertApp] Writing 6379 bytes to data/script630.py
[NbConvertApp] Converting notebook data/script631.ipynb to python
[NbConvertApp] Writing 17133 bytes to data/script631.py
[NbConvertApp] Converting notebook data/script632.ipynb to python
[NbConvertApp] Writing 34951 bytes to data/script632.py
[NbConvertApp] Converting notebook data/script633.ipynb to python
[NbConvertApp] Writing 6535 bytes to data/script633.py
[NbConvertApp] Converting notebook data/script634.ipynb to python
[NbConvertApp] Writing 18022 bytes to data/script634.py
[NbConvertApp] Converting notebook data/script635.ipynb to python
[NbConvertApp] Writing 10292 bytes to data/script635.py
[NbConvertApp] Converting notebook data/script636.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('outputs', 'execution_count' were unexpected)

Failed validati

[NbConvertApp] Converting notebook data/script675.ipynb to python
[NbConvertApp] Writing 17538 bytes to data/script675.py
[NbConvertApp] Converting notebook data/script676.ipynb to python
[NbConvertApp] Writing 9219 bytes to data/script676.py
[NbConvertApp] Converting notebook data/script677.ipynb to python
[NbConvertApp] Writing 3454 bytes to data/script677.py
[NbConvertApp] Converting notebook data/script678.ipynb to python
[NbConvertApp] Writing 11784 bytes to data/script678.py
[NbConvertApp] Converting notebook data/script679.ipynb to python
[NbConvertApp] Writing 1298 bytes to data/script679.py
[NbConvertApp] Converting notebook data/script68.ipynb to python
[NbConvertApp] Writing 8534 bytes to data/script68.py
[NbConvertApp] Converting notebook data/script680.ipynb to python
[NbConvertApp] Writing 2823 bytes to data/script680.py
[NbConvertApp] Converting notebook data/script681.ipynb to python
[NbConvertApp] Writing 12556 bytes to data/script681.py
[NbConvertApp] Converting noteb

In [86]:
methods_write = open('methods.txt', 'w')
for method in all_methods:
    methods_write.write("%s\n" % method)

In [87]:
print(len(all_methods))

932


In [84]:
base = {}
for dict_method in all_methods:
    for k in dict_method:
        if k not in base:
            base[k] = 0 
        base[k] += dict_method[k]

In [85]:
m, c = [], []
for method, count in base.items():
    m.append(method)
    c.append(count)
    
mFreq = pd.DataFrame(data = {'Method' : m, 'Count': c}).set_index('Method').sort_values('Count', ascending=False)
mFreq.to_csv('results.csv')
mFreq

,Count
Method,
pd.read_csv,756
pd.DataFrame,469
df.head,438
df.append,436
df.mean,429
df.drop,422
df.sum,414
df.ge,401
df.to_csv,378
